# Lab 8: Greedy Algorithms

Lab associated with Module 8: Greedy Algorithms

***

In [65]:
# The following lines are used to increase the width of cells to utilize more space on the screen 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

C:\Users\benny\AppData\Local\Temp\ipykernel_57508\1945220536.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


***

### Section 0: Imports

In [66]:
import numpy as np

In [67]:
import math

In [68]:
from IPython.display import Image
from graphviz import Digraph

***

### <font color='red'> Activity 1a: Write a greedy algorithm for Activity Selection problem in this week's lab. </font>

![Union is appending](Union-is-appending.png)

### Realised that Union is Appending to the list, not adding to the set

In [82]:
# activity selection algorithm
# activites a1, a2 ... an
# start time s1, s2 ... sn
# finish time f1, f2 ... fn
# we have to do n activities and each activity has to be started at a specified time and finished at specified time
# if the activities overlap, can we devise an algorithm to come-up with an order on the activities to do as many activities as possible
# The question is which activities should we choose and which should we skip
# sorted finish time is O(n) otherwise O(nlogn)
# each step is locally optimal

def greedyActivitySelection(activites):
    if len(activites) == 0:
        return []
    sortedActivities = sorted(activites, key=lambda x: x[1])
    selectedActivities = [sortedActivities[0]]
    for i in range(1, len(sortedActivities)):
        if sortedActivities[i][0] >= selectedActivities[-1][1]:
            selectedActivities.append(sortedActivities[i])
    return selectedActivities









In [83]:
import unittest

class TestGreedyActivitySelection(unittest.TestCase):
    
    def test_empty_activities(self):
        activities = []
        expected_output = []
        self.assertEqual(greedyActivitySelection(activities), expected_output)
        
    def test_single_activity(self):
        activities = [(1, 2)]
        expected_output = [(1, 2)]
        self.assertEqual(greedyActivitySelection(activities), expected_output)
        
    def test_disjoint_activities(self):
        activities = [(1, 2), (3, 4), (5, 6)]
        expected_output = [(1, 2), (3, 4), (5, 6)]
        self.assertEqual(greedyActivitySelection(activities), expected_output)
        
    def test_overlapping_activities(self):
        activities = [(1, 3), (2, 4), (3, 5), (4, 6)]
        expected_output_1 = [(1, 3), (3, 5)]
        expected_output_2 = [(2, 4), (4, 6)]
        result = greedyActivitySelection(activities)
        self.assertTrue(result in [expected_output_1, expected_output_2])

unittest.main(argv=[''], exit=False)

F....
FAIL: test_all_activities_overlap (__main__.TestGreedyActivitySelection.test_all_activities_overlap)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\benny\AppData\Local\Temp\ipykernel_57508\2810846043.py", line 33, in test_all_activities_overlap
    self.assertEqual(len(result), expected_output)
AssertionError: 2 != [(1, 5)]

----------------------------------------------------------------------
Ran 5 tests in 0.002s

FAILED (failures=1)


[(2, 4), (4, 7)]


In [77]:
activities = [  [1,4],[2,5],[3,6],[5,7],[3,8],[6,9],[8,10],[9,11],[5,12],[6,13],[8,14],[13,15] ]

activityList = greedyActivitySelection(activities)

print("Solution:")
for activity in activityList:
    print(activity)

Solution:
[1, 4]
[5, 7]
[8, 10]
[13, 15]


### <font color='red'> Activity 1b: Devise a dynamic programming solution to Activity Selection problem, and see if you get the same results as that of greedy strategy. </font>

In [72]:
#### TODO ####
### Good Luck ###







Let us test above function as:

In [73]:
activities = [  [1,4],[2,5],[3,6],[5,7],[3,8],[6,9],[8,10],[9,11],[5,12],[6,13],[8,14],[13,15] ]

activityList = dynamicProgrammingActivitySelection(activities)

print("Solution:")
for act in activityList:
    print(activities[act])

NameError: name 'dynamicProgrammingActivitySelection' is not defined

***

###  Prim's Algorithm

Graph's Preliminaries

In [ ]:
from graph import *

In [ ]:
G = Graph()

for i in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']:
    G.addVertex( Node(i) )

In [ ]:
V = G.vertices

#0, 1, 2, 3, 4, 5, 6, 7, 8
#A, B, C, D, E, F, G, H, I

G.addBiEdge( V[0], V[1], 4)

G.addBiEdge( V[0], V[7] , 8)

G.addBiEdge( V[1], V[7], 11)

G.addBiEdge( V[1], V[2], 8)

G.addBiEdge( V[2], V[3], 7)

G.addBiEdge( V[3], V[4], 9)

G.addBiEdge( V[3], V[5], 14 )

G.addBiEdge( V[4], V[5], 10 )

G.addBiEdge( V[2], V[5], 4 )

G.addBiEdge( V[2], V[8], 2 )

G.addBiEdge( V[5], V[6], 2 )

G.addBiEdge( V[6], V[7], 1 )

G.addBiEdge( V[6], V[8], 6 )

G.addBiEdge( V[7], V[8], 7 )


In [ ]:
print(G)

This is what we had in the lectures as the slow implementation of Prim's Algorithm

In [ ]:
# G is graph
# s is the node to start

def slowPrim(G, s):
    
    # first, find the lightest edge leaving s
    bestWt = np.inf
    bestu = None
    
    for u,wt in s.getOutNeighborsWithWeights():
        
        if wt < bestWt:
            bestWt = wt
            bestu = u
    
    MST = [ (s, bestu) ]
    verticesVisited = [s,bestu]
    
    while len(verticesVisited) < len(G.vertices): # danger! this will loop forever if the graph isn't connected...
        
        # find the lightest edge (x,v) so that x has been visited and v hasn't.
        bestWt = np.inf
        bestv = None
        bestx = None
        
        for x in verticesVisited:
            for v,wt in x.getOutNeighborsWithWeights():
                
                if v in verticesVisited:
                    continue
                
                if wt < bestWt:
                    bestWt = wt
                    bestv = v
                    bestx = x
                    
        MST.append((bestx,bestv))
        verticesVisited.append(bestv)
    
    return MST

In [ ]:
T = slowPrim(G, G.vertices[0])

for x,y in T:
    print(x,y)

Okay, it seems to be working fine, but as we discussed, will be quite slow. Let us see if we can work on the faster version of the code as:

### <font color='red'> Activity 2: In lights of Prim's Algorithm above, write an efficient implementation based on our discussions in the Seminar/Lecture. </font>

In [ ]:
def prim(G,w):
    
    #### TODO ####
    ### Good Luck ###
                    
    return MST

In [ ]:
T = prim(G, G.vertices[0])

for x,y in T:
    print(x,y)

***